Version 5: hyperparameter tuning using GPU

Version 11: hyperparameter tuning using CPU because we can't get reproducible results using GPU caused by GPU double precision problem [Find by @chenjiexu]


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import random
import optuna
import time

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/icr-create-folds/train_folds.csv')

In [3]:
df['EJ'] = df['EJ'].replace({'A': 0, 'B': 1})
df = df.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})

In [4]:
features = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', #'BC', 
            'BD', 'BN', 'BP', 'BQ', 'BR', 'BZ',
            'CB', 'CC', 'CD', 'CF', 'CH', #'CL', 
            'CR', 'CS', 'CU', 'CW',
            'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
            'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU',
            'FC', 'FD', 'FE', 'FI', 'FL', 'FR', 'FS',
            'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'EJ']
label    = df.columns[-2]

In [5]:
def balance_logloss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred / np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    
    logloss = (-w0/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - w1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / (w0+w1)
    
    return logloss

In [6]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(42)

In [7]:
def calc_log_loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

NB: I use all 5 folds to not overfit one fold

In [8]:
# Set the maximum running time (in seconds) to 11h hours and 40 minutes because the seasion of kaggle in 12h using CPU
max_running_time = 40 * 60 + 11 * 60 * 60 

# Start the timer
start_time = time.time()

def run(trial):
    final_valid_predictions = {}
    bs = []
    
    learning_rate    = trial.suggest_float("learning_rate", 1e-3, 0.1)
    boosting         = trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss'])
    num_leaves       = trial.suggest_int("num_leaves", 2, 40)
    feature_fraction = trial.suggest_uniform("feature_fraction", 0.2, 1.0)
    bagging_fraction = trial.suggest_uniform("bagging_fraction", 0.2, 1.0)
    lambda_l1        = trial.suggest_loguniform('lambda_l1', 1e-8, 10.0)
    lambda_l2        = trial.suggest_loguniform('lambda_l2', 1e-8, 10.0)
    
    for k in range(5):
        print('------------------ Fold: '+str(k))
        train     = df[df['kfold'] !=k].reset_index(drop=True)
        val       = df[df['kfold'] ==k].reset_index(drop=True)
        valid_ids = val.Id.values.tolist()

        train_w0, train_w1 = calc_log_loss_weight(train[label])
        valid_w0, valid_w1 = calc_log_loss_weight(val[label])

        train_dataset = lgb.Dataset(train[features], train[label],weight=train[label].map({0: train_w0, 1: train_w1}), categorical_feature=["EJ"] )
        eval_dataset  = lgb.Dataset(val[features], val[label], weight=val[label].map({0: valid_w0, 1: valid_w1}), categorical_feature=["EJ"])
        lgb_params = {
            'objective': 'binary',
            'metric': 'binary_logloss', 
            'boosting': boosting,
            'learning_rate': learning_rate,
            'num_leaves': num_leaves,
            'feature_fraction': feature_fraction,
            'bagging_fraction': bagging_fraction,
            'lambda_l1': lambda_l1, 
            'lambda_l2': lambda_l2,
            'n_jobs': -1,
            'is_unbalance':True, 
            'verbose': -1,
            'seed': 42,
        }

        model = lgb.train(
                    params = lgb_params,
                    train_set = train_dataset,
                    num_boost_round = 50000,
                    valid_sets = [train_dataset, eval_dataset],
                    early_stopping_rounds = 20,
                    verbose_eval = 10000,
                )

        preds_valid = model.predict(val[features])
        preds_valid = np.vstack([1 - preds_valid, preds_valid]).T

        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
        blogloss = balance_logloss(val[label], preds_valid)

        bs.append(blogloss)
        print(k, blogloss)
    print('Balance Log loss:')
    print(bs)
    print(np.mean(bs), np.std(bs))
    
    final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
    final_valid_predictions.columns = ['Id', 'class_0', 'class_1']
    final_valid_predictions.to_csv(r"oof.csv", index=False)

    #Befor checking accuracy let's first make sure that we have the data are sorted in the same way :)
    final_valid_predictions   = final_valid_predictions.sort_values('Id')
    df_sorted                 = df.sort_values('Id')
    final_score               = balance_logloss(df_sorted['Class'], final_valid_predictions[['class_0', 'class_1']].values)
    print('Final score of this trial is: '+str(final_score))
    return final_score

In [9]:
%%time
study = optuna.create_study(direction="minimize")
while True:
    # Check if the maximum running time has been exceeded
    elapsed_time = time.time() - start_time
    if elapsed_time > max_running_time:
        break
    
    # Optimize a single trial
    study.optimize(run, n_trials=1)
    
    # Print intermediate results
    print('Best trial:', study.best_trial.params)
    print('Current elapsed time:', elapsed_time)
    
# Get the best hyperparameters
best_params = study.best_trial.params
print('Best hyperparameters:', best_params)


[I 2023-06-03 13:52:07,651] A new study created in memory with name: no-name-121f3efe-acff-4497-9e22-2152205ec2fb


------------------ Fold: 0
[10000]	training's binary_logloss: 3.476e-05	valid_1's binary_logloss: 0.16371
[20000]	training's binary_logloss: 2.12307e-06	valid_1's binary_logloss: 0.220025
[30000]	training's binary_logloss: 2.13762e-06	valid_1's binary_logloss: 0.196411
[40000]	training's binary_logloss: 2.00833e-06	valid_1's binary_logloss: 0.211366
[50000]	training's binary_logloss: 2.02029e-06	valid_1's binary_logloss: 0.209513
0 0.31650382748312555
------------------ Fold: 1
[10000]	training's binary_logloss: 1.55317e-05	valid_1's binary_logloss: 0.810607
[20000]	training's binary_logloss: 2.14092e-06	valid_1's binary_logloss: 0.846133
[30000]	training's binary_logloss: 2.12969e-06	valid_1's binary_logloss: 0.846797
[40000]	training's binary_logloss: 1.99223e-06	valid_1's binary_logloss: 0.848479
[50000]	training's binary_logloss: 1.94712e-06	valid_1's binary_logloss: 0.875398
1 1.3559873279069254
------------------ Fold: 2
[10000]	training's binary_logloss: 1.82779e-05	valid_1's bi

[I 2023-06-03 13:57:55,061] Trial 0 finished with value: 1.006850100665028 and parameters: {'learning_rate': 0.03749692818227844, 'boosting': 'dart', 'num_leaves': 3, 'feature_fraction': 0.5698265778990128, 'bagging_fraction': 0.8756043886117362, 'lambda_l1': 3.3468356238232837e-07, 'lambda_l2': 0.001933971550428488}. Best is trial 0 with value: 1.006850100665028.


4 1.6480700465989784
Balance Log loss:
[0.31650382748312555, 1.3559873279069254, 1.0605404267944312, 0.6668002684945037, 1.6480700465989784]
1.0095803794555929 0.47499832984412754
Final score of this trial is: 1.006850100665028
Best trial: {'learning_rate': 0.03749692818227844, 'boosting': 'dart', 'num_leaves': 3, 'feature_fraction': 0.5698265778990128, 'bagging_fraction': 0.8756043886117362, 'lambda_l1': 3.3468356238232837e-07, 'lambda_l2': 0.001933971550428488}
Current elapsed time: 0.019565582275390625
------------------ Fold: 0
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[423]	training's binary_logloss: 0.0186668	valid_1's binary_logloss: 0.0970356
0 0.10101480705770441
------------------ Fold: 1
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[197]	training's binary_logloss: 0.0980897	valid_1's binary_logloss: 0.315405
1 0.2879707630347565
------------------ Fold: 2
Training until v

[I 2023-06-03 13:57:56,286] Trial 1 finished with value: 0.19833581016407026 and parameters: {'learning_rate': 0.031212557705969785, 'boosting': 'goss', 'num_leaves': 31, 'feature_fraction': 0.8766220304922052, 'bagging_fraction': 0.8697399463067232, 'lambda_l1': 1.8157409883726593e-08, 'lambda_l2': 5.8058051931061435e-06}. Best is trial 1 with value: 0.19833581016407026.


...

Early stopping, best iteration is:
[298]	training's binary_logloss: 0.0234773	valid_1's binary_logloss: 0.188645
3 0.16097153629295638
------------------ Fold: 4
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[158]	training's binary_logloss: 0.0942318	valid_1's binary_logloss: 0.284637
4 0.2897532106781924
Balance Log loss:
[0.10198038469214998, 0.29618735797697554, 0.2041592081515037, 0.16097153629295638, 0.2897532106781924]
0.21061033955835562 0.074690180225777
Final score of this trial is: 0.21045354781858047
Best trial: {'learning_rate': 0.07304687551137469, 'boosting': 'goss', 'num_leaves': 7, 'feature_fraction': 0.5201835814022956, 'bagging_fraction': 0.6198542622273945, 'lambda_l1': 0.00025103322186762747, 'lambda_l2': 1.9523346562250452}
Current elapsed time: 41822.88054704666
------------------ Fold: 0
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[174]	training's binary_logloss:

[I 2023-06-04 01:29:12,054] Trial 1444 finished with value: 0.20768407254694574 and parameters: {'learning_rate': 0.08618664904496003, 'boosting': 'goss', 'num_leaves': 7, 'feature_fraction': 0.6891982317034545, 'bagging_fraction': 0.25033929994314935, 'lambda_l1': 0.6150419752912695, 'lambda_l2': 7.910648959037113e-06}. Best is trial 1276 with value: 0.17766296507671236.


Early stopping, best iteration is:
[190]	training's binary_logloss: 0.0151576	valid_1's binary_logloss: 0.160278
3 0.11401676704772742
------------------ Fold: 4
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[83]	training's binary_logloss: 0.0954712	valid_1's binary_logloss: 0.290409
4 0.29435538932402566
Balance Log loss:
[0.09410743841597699, 0.3145449272940587, 0.2195223577117373, 0.11401676704772742, 0.29435538932402566]
0.20730937595870524 0.0902705027981312
Final score of this trial is: 0.20768407254694574
Best trial: {'learning_rate': 0.07304687551137469, 'boosting': 'goss', 'num_leaves': 7, 'feature_fraction': 0.5201835814022956, 'bagging_fraction': 0.6198542622273945, 'lambda_l1': 0.00025103322186762747, 'lambda_l2': 1.9523346562250452}
Current elapsed time: 41823.71455979347
------------------ Fold: 0
[10000]	training's binary_logloss: 2.42636e-06	valid_1's binary_logloss: 0.181418
[20000]	training's binary_logloss: 1.90405e-0

[I 2023-06-04 01:37:13,030] Trial 1445 finished with value: 1.3856827305518207 and parameters: {'learning_rate': 0.06263415399604066, 'boosting': 'dart', 'num_leaves': 9, 'feature_fraction': 0.9001719869026199, 'bagging_fraction': 0.4502681185249716, 'lambda_l1': 6.838964316886225e-07, 'lambda_l2': 0.000936595490802809}. Best is trial 1276 with value: 0.17766296507671236.


4 2.0061534169205695
Balance Log loss:
[0.43617095591012134, 2.1428562246512923, 1.5235974867054785, 0.8175649843105091, 2.0061534169205695]
1.385268613699594 0.6635729920797458
Final score of this trial is: 1.3856827305518207
Best trial: {'learning_rate': 0.07304687551137469, 'boosting': 'goss', 'num_leaves': 7, 'feature_fraction': 0.5201835814022956, 'bagging_fraction': 0.6198542622273945, 'lambda_l1': 0.00025103322186762747, 'lambda_l2': 1.9523346562250452}
Current elapsed time: 41824.42503762245
Best hyperparameters: {'learning_rate': 0.07304687551137469, 'boosting': 'goss', 'num_leaves': 7, 'feature_fraction': 0.5201835814022956, 'bagging_fraction': 0.6198542622273945, 'lambda_l1': 0.00025103322186762747, 'lambda_l2': 1.9523346562250452}
CPU times: user 21h 30min 54s, sys: 1h 3min 3s, total: 22h 33min 57s
Wall time: 11h 45min 5s
